![Bootcamp Microsoft Certification Challenge #1 - AI 102](https://assets.dio.me/79IKKjY5EHRPqlscNsYum7Iv9uQNa_siSO7Ab8Zv3II/f:webp/h:120/q:80/L3RyYWNrcy8wMzI1ZTE2Ni1mYTNjLTRmMWItYWNlMS04ZTdmM2M0ZDU4NDEucG5n)

# Desafio de projeto: Tradutor de Artigos Técnicos com AzureAI

Desenvolvendo uma solução de tradução de documentos e artigos utilizando o AzureAI.

### Instalação e importação das libs necessárias

In [2]:
!pip install beautifulsoup4 requests python-docx  openai langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13


In [10]:
import os
import requests
from bs4 import BeautifulSoup
from docx import Document
from langchain_openai.chat_models.azure import AzureChatOpenAI
from google.colab import userdata

## 1 - Azure Translate

In [ ]:
endpoint = 'https://api.cognitive.microsofttranslator.com'
subscription_key = userdata.get('AZURE_TRANSLATE_SUBSCRIPTION_KEY')
location = 'location'
original_language = 'en-us'
target_language = 'pt-br'

In [ ]:
def make_translation_request(text, from_language, original_language, endpoint):

    # Definir o caminho para o endpoint de tradução.
    path = '/translate'
    url = endpoint + path

    # Configurar os cabeçalhos necessários para autenticação e tipo de conteúdo
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,   # Chave da API
        'Ocp-Apim-Subscription-Region': location,        # Região
        'Content-type': 'application/json',              # Tipo de conteúdo (JSON)
        'X-ClientTraceId': str(os.urandom(16))           # Identificador único
    }

    # Corpo da solicitação, contendo o texto que será traduzido
    body = [{'text': text}]

    # Parâmetros da URL, definindo a versão da API e os idiomas de origem e destino
    params = {
        'api-version': '3.0',
        'from': original_language,
        'to': target_language
    }

    # Enviar a solicitação para o endpoint de tradução
    request = requests.post(url, params=params, headers=headers, json=body)

    # Retornar a resposta em formato JSON com o resultado da tradução
    return request.json()


def translate_text(text, target_language='pt-br', original_language='en'):
    # Chama a função make_translation_request para fazer a tradução do texto
    response = make_translation_request(text, original_language, target_language)

    # Extrai e retorna o texto traduzido da resposta JSON da API
    return response[0]['translations'][0]['text']


def translate_document(path, target_language='pt-br', original_language='en'):
    # Carrega o documento
    doc = Document(path)
    # Junta todos os parágrafos em um único texto
    full_text = "\n".join(paragraph.text for paragraph in doc.paragraphs)

    # Realiza a tradução fazendo uso da função make_translation_request
    response = make_translation_request(full_text, original_language, target_language)
    translated_text = response[0]['translations'][0]['text']

    # Cria um novo documento e adiciona o texto traduzido
    translated_doc = Document()
    for line in translated_text.split("\n"):
        translated_doc.add_paragraph(line)

    # Salva o documento traduzido
    path_translated = path.replace(".docx", f"_{target_language}.docx")
    translated_doc.save(path_translated)

    return path_translated

In [ ]:
# Testando o Azure Translate com uma estrofe da música Walk

estrofe = """A million miles away
Your signal in the distance
To whom it may concern
I think I lost my way
Getting good at starting over
Every time that I return"""

estrofe_traduzida = translate_text(estrofe)
print(estrofe_traduzida)

In [ ]:
# Testando o Azure Translate com o documento com a música Walk

document_path = '/content/walk.docx'
translated_document_path = translate_document(document_path)
print(translated_document_path)

musica = Document(translated_document_path)
musica = "\n".join(paragraph.text for paragraph in musica.paragraphs)
print()
print(musica)

## 2- Azure OpenAI

In [11]:
def extract_text_from_url(url):
    """
    Extrai e limpa o texto de uma página web fornecida por meio de uma URL.

    """

    # Requisição GET na URL passada
    response = requests.get(url)

    # Verifica se a requisição foi bem-sucedida (código de status 200)
    if response.status_code == 200:
        # Pega o conteúdo da página
        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove elementos 'script' e 'style'
        for script_or_style in soup(["script", "style"]):
            script_or_style.decompose()

        # Extrai o texto da página, usando espaços como separador entre blocos
        text = soup.get_text(separator=' ')

        # Limpa o texto para remover espaços e linhas extras
        lines = (line.strip() for line in text.splitlines())  # Remove espaços ao redor de cada linha
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))  # Remove espaços internos duplos

        # Reúne o texto limpo em blocos, separados por quebras de linha
        texto_limpo = '\n'.join(chunk for chunk in chunks if chunk)

    else:
        # Se der erro, printa o código de status HTTP
        print(f"Failed to fetch URL. Status code: {response.status_code}")
        return None

    # Retorna o texto limpo extraído da página
    return texto_limpo


def translate_article(text, lang, client):
    """
    Traduz um artigo para o idioma especificado utilizando um cliente de tradução.

    """

    # Define mensagens para o cliente
    messages = [
        ("system", "Você atua como tradutor de textos"),
        ("user", f"Traduza o {text} para o idioma {lang} e dê a resposta no formato markdown")
    ]

    # Envia as mensagens para o cliente e obtém a resposta com o texto traduzido
    response = client.invoke(messages)

    # Retorna o conteúdo traduzido da resposta
    return response.content


In [ ]:
url = 'https://dev.to/modernsystemdesign/llm-for-dummies-3h90'

azure_endpoint = ''
azure_openai_api_key = userdata.get('AZURE_OPENAI_API_KEY')
api_version = '2023-05-15'
deployment_name = 'gpt-4o-mini'


client = AzureChatOpenAI(
  azure_endpoint=azure_endpoint,
  api_key=azure_openai_api_key,
  api_version=api_version,
  deployment_name=deployment_name,
  max_retries=0
)


text = extract_text_from_url(url)
translated_article = translate_article(text, "pt-br", client)

print(translated_article)